In [1]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import re

In [24]:
soup = BeautifulSoup(requests.get('https://ru.kinorium.com/movies/online/?order=rating&page=3&perpage=200&show_viewed=1').text, "html.parser")
print(soup.prettify())

<!DOCTYPE html >
<html lang="ru">
 <head prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns/fb# video: http://ogp.me/ns/video# profile: http://ogp.me/ns/profile#">
  <title>
   Премьеры - Онлайн
  </title>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <link href="//static-s.kinorium.com" rel="dns-prefetch"/>
  <link href="//images-s.kinorium.com" rel="dns-prefetch"/>
  <link href="//www.gstatic.com" rel="dns-prefetch"/>
  <link href="//ru-images-s.kinorium.com" rel="dns-prefetch"/>
  <link href="//static-s.kinorium.com" rel="preconnect"/>
  <link href="//images-s.kinorium.com" rel="preconnect"/>
  <link href="//ru-images-s.kinorium.com" rel="preconnect"/>
  <link href="//www.gstatic.com" rel="preconnect"/>
  <link as="style" href="//static-s.kinorium.com/css/min/style-default.css?1687953253" rel="preload"/>
  <link as="script" crossorigin="crossorigin" href="//static-s.kinorium.com/js/min/vendor.min.js?1687953253" rel="preload"/>
  <link as="script" crossorig

In [25]:
name = soup.find('i', class_ = 'movie-title__text').text
name

' Шпион, которого не было '

In [29]:
link = 'https://ru.kinorium.com' + soup.find('a', class_ = 'filmList__item-title').get('href')
link

'https://ru.kinorium.com/2431187/'

In [58]:
soup = BeautifulSoup(requests.get('https://ru.kinorium.com/2431187/').text, "html.parser")

In [54]:
countries = []
for country in soup.findAll('a', class_ = 'film-page__country-link'):
    countries.append(country.text)
print(countries)

['Великобритания', 'Германия', 'США']


In [53]:
genres = []
for genre in soup.findAll('li', itemprop = 'genre'):
    genres.append(genre.text)
print(genres)

['Драма', 'Триллер']


In [65]:
soup = BeautifulSoup(requests.get('https://ru.kinorium.com/2431187/cast').text, "html.parser")

In [84]:
all_cast = []
for cast in soup.findAll('a', class_ = 'cast-page__tags-link'):
    all_cast.append(cast.text)
print(all_cast)

[' Режиссёр ', ' Актёры ', ' Сценарист ', ' Автор ', ' Продюсеры ', ' Оператор ', ' Композитор ', ' Звукорежиссёры ', ' Художники ', ' Монтажёр ', ' Режиссёр дубляжа ', ' Дубляж ', ' Переводчик ']


In [74]:
for name in soup.findAll('h5', class_ = 'cast-page__item-name'):
    print(name.text)

Адам Паттерсон 
Джеймс Нортон 
Джемма Артертон 
Мэттью Дуглас 
Роб Мэлоун 
Мариса Абела 
Фрейя Мавор 
Питер Хинан 
Майкл Дрэйк 
Джулиан Бэррэтт 
Филип Райт 
Джимми Экинбола 
Майкл Фентон Стивенс 
Шарлотта Эйвери 
Саймон Чэндлер 
Мелисса Колье 
Шазад Латиф 
Люк Пьер 
Сара Голдберг 
Эдвина Финдли 
Кэти Федерман 
Тянь Чодри 
Лин Галлахер 
Мартин Уолш 
Стивен Амис 
Марк Локхарт 
Робби Майн 
Селина Несса 
Крис Уилсон 
Адам Паттерсон 
Майкл Броннер 
Майкл Броннер 
Джина Картер 
Kitty Kaletsky 
Роберт Тейлор 
Херберт Клойбер 
Джеймс Нортон 
Роберт Хальми 
Оливия Паль 
Адам Паттерсон 
Джим Рив 
Харольд ван Лир 
Джон Палфери Смит 
Лаура Бэйкер 
Ларри Смит 
Ханнал Пил 
Мэтт Дэниэл 
Саймон Хилл 
Уилл Уэли 
Чарльз Де Ла Мейр 
Кэтрин Мередидд 
Джо Дело 
Эмили-Роуз Яксис 
Мэгс Арнольд 
Владимир Рыбальченко 
Антон Савенков 
Екатерина Семенова 
Георгий Кармрян 
Дина Бобылёва 
Анастасия Лапина 
Дмитрий Лунёв 
Владимир Рыбальченко 
Антон Морозов 
Сергей Пономарёв 
Антон Морозов 
Алексей Войтюк 
Пётр Ива